# Criar Corpus Específico do CohQuAD Coh pt-br

Geração do corpus específico para o conjunto de dados.


Gera as POS-Tagging dos documentos perturbados do conjunto de dados.
- Utiliza dados textos sobre o assunto do conjunto de dados.
- Gera o arquivo `corpus_especifico.zip`.
 


# 1 Preparação do ambiente

Preparação do ambiente para execução do script.

## 1.1 Tempo inicial de processamento

In [ ]:
# Import das bibliotecas.
import time
import datetime

# Marca o tempo de início do processamento
inicio_processamento = time.time()

## 1.2 Funções e classes auxiliares

Verifica se existe o diretório cohebert no diretório corrente.   


In [ ]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================  
def verificaDiretorioCoheBERT():
    """
      Verifica se existe o diretório cohebert no diretório corrente.    
    """
    
    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_COHEBERT):  
        # Cria o diretório
        os.makedirs(DIRETORIO_COHEBERT)
        logging.info("Diretório Cohebert criado: {}".format(DIRETORIO_COHEBERT))
    
    return DIRETORIO_COHEBERT

Realiza o download e um arquivo

In [ ]:
# Import das bibliotecas.
import requests # Biblioteca de download
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso
import os # Biblioteca para manipular arquivos

def downloadArquivo(url_arquivo, nome_arquivo_destino):
    """
      Realiza o download de um arquivo de uma url em salva em nome_arquivo_destino.
    
      Parâmetros:
        `url_arquivo` - URL do arquivo a ser feito download.      
        `nome_arquivo_destino` - Nome do arquivo a ser salvo.      
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Realiza o download de um arquivo em uma url
    data = requests.get(url_arquivo, stream=True)
    
    # Verifica se o arquivo existe
    if data.status_code != 200:
        logging.info("Exceção ao tentar realizar download {}. Response {}.".format(url_arquivo, data.status_code))
        data.raise_for_status()
        return

    # Recupera o nome do arquivo a ser realizado o download    
    nome_arquivo = nome_arquivo_destino.split("/")[-1]  

    # Define o nome e caminho do arquivo temporário    
    nome_arquivo_temporario = DIRETORIO_COHEBERT + "/" + nome_arquivo + "_part"
    
    logging.info("Download do arquivo: {}.".format(nome_arquivo_destino))
    
    # Baixa o arquivo
    with open(nome_arquivo_temporario, "wb") as arquivo_binario:        
        tamanho_conteudo = data.headers.get("Content-Length")        
        total = int(tamanho_conteudo) if tamanho_conteudo is not None else None
        # Barra de progresso de download
        progresso_bar = tqdm_notebook(unit="B", total=total, unit_scale=True)                
        # Atualiza a barra de progresso
        for chunk in data.iter_content(chunk_size=1024):        
            if chunk:                
                progresso_bar.update(len(chunk))
                arquivo_binario.write(chunk)
    
    # Renomeia o arquivo temporário para o arquivo definitivo
    os.rename(nome_arquivo_temporario, nome_arquivo_destino)
    
    # Fecha a barra de progresso.
    progresso_bar.close()

Remove tags de um documento

In [ ]:
def remove_tags(documento):
    """
      Remove tags de um documento
    """
    
    import re

    documento_limpo = re.compile("<.*?>")
    return re.sub(documento_limpo, "", documento)

Funções auxiliares de arquivos

In [ ]:
def carregar(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como um único parágrafo(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.  
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    paragrafo = ""
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        # Remove as tags existentes no final das linhas
        linha = remove_tags(linha)
        if linha != "":
          paragrafo = paragrafo + linha.strip() + " "
    
    # Fecha o arquivo
    arquivo.close()

    # Remove os espaços em branco antes e depois do parágrafo
    return paragrafo.strip()

In [ ]:
def carregarLista(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como uma lista de sentenças(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.   
        `encoding` - Codificação dos caracteres do arquivo.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    sentencas = []
    for linha in arquivo:        
        linha = linha.splitlines()
        linha = " ".join(linha)
        linha = remove_tags(linha)
        if linha != "":
          sentencas.append(linha.strip())
    
    # Fecha o arquivo
    arquivo.close()

    return sentencas 

In [ ]:
def salvar(nome_arquivo,texto):                       
    """
      Salva um texto em arquivo.
     
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser salvo.
        `texto` - Texto a ser salvo.     
    """

    arquivo = open(nome_arquivo, "w")
    arquivo.write(str(texto))
    arquivo.close()

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))    

Classe(ModelArguments) de definição dos parâmetros do modelo

In [ ]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosMedida:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={"help": "max seq len"},
    )    
    pretrained_model_name_or_path: str = field(
        default="neuralmind/bert-base-portuguese-cased",
        metadata={"help": "nome do modelo pré-treinado do BERT."},
    )
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )
    versao_modelo_spacy: str = field(
        default="-3.2.0",
        metadata={"help": "versão do nome do modelo no spaCy."},
    )
    sentenciar_documento: bool = field(
        default=True,
        metadata={"help": "Dividir o documento em sentenças(frases)."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={"help": "define se o texto do modelo deve ser todo em minúsculo."},
    )    
    output_attentions: bool = field(
        default=False,
        metadata={"help": "habilita se o modelo retorna os pesos de atenção."},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={"help": "habilita gerar as camadas ocultas do modelo."},
    )
    usar_mcl_ajustado : bool = field(
        default=False,
        metadata={"help": "habilita o carragamento de mcl ajustado."},
    )
    documentos_perturbados: int = field(
        default="1",
        metadata={"help": "Quantidade de documentos a serem perturbados a partir do original."},
    )
    top_k_predicao: int = field(
        default="100",
        metadata={"help": "Quantidade de palavras a serem recuperadas mais próximas da máscara."},
    )    

Biblioteca de limpeza de tela


In [ ]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [ ]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s")

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 1.4 Identificando o ambiente Colab

In [ ]:
# Import das bibliotecas.
import sys # Biblioteca para acessar módulos do sistema

# Se estiver executando no Google Colaboratory
# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [ ]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:root:Dispositivo GPU não encontrado


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [ ]:
# Import das bibliotecas.
import torch

def getDeviceGPU():
    """
      Retorna um dispositivo de GPU se disponível ou CPU.
    
      Retorno:
        `device` - Um device de GPU ou CPU.       
    """
        
    # Se existe GPU disponível.
    if torch.cuda.is_available():
        
        # Diz ao PyTorch para usar GPU.    
        device = torch.device("cuda")
        
        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:        
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")
        
    return device

In [ ]:
# Recupera o device com GPU ou CPU
device = getDeviceGPU()

INFO:root:Sem GPU disponível, usando CPU.


Memória

Memória disponível no ambiente

In [ ]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

INFO:root:Seu ambiente de execução tem  13.6 gigabytes de RAM disponível

INFO:root:Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
INFO:root:e selecione High-RAM. Então, execute novamente está célula


## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.

In [ ]:
# import necessário
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount("/content/drive")

Mounted at /content/drive


## 1.7 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [ ]:
# Instala o spacy
!pip install -U pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 52.4 MB/s 
  Attempting uninstall: wheel
    Found existing installation: wheel 0.38.3
    Uninstalling wheel-0.38.3:
      Successfully uninstalled wheel-0.38.3
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [ ]:
# Instala uma versão específica
!pip install -U spacy==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.6/660.6 kB 37.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.2
    Uninstalling pydantic-1.10.2:
      Successfully uninstalled pydantic-1.10.2
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.5
    Uninstalling thinc-8.1.5:
      Successfully uninstalled thinc-8.1.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.2
    Uninstalling spacy-3.4.2:
      Successfully uninstalled spacy-3.4.2
ERROR

## 1.8 Instalação do Gensim

Instalando o gensim no Google Colaboratory.

No Jupiter Notebook executar através "Anaconda Prompt".


In [ ]:
#!pip install -U gensim
!pip install -U gensim==4.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 54.8 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


# 2 Parametrização

## Gerais

## Específicos

Parâmetros do modelo

In [ ]:
# Definição dos parâmetros do Modelo.
model_args = ModeloArgumentosMedida(     
    modelo_spacy = "pt_core_news_lg",
    #modelo_spacy = "pt_core_news_md",
    #modelo_spacy = "pt_core_news_sm",

    versao_modelo_spacy = "3.2.0",
    
    sentenciar_documento = True,
    do_lower_case = False, # default True  
)

## Nome do diretório dos arquivos de dados

In [ ]:
# Diretório do cohebert
DIRETORIO_COHEBERT = "COHQUAD_CO_PTBR"

## Define o caminho para os arquivos de dados

In [ ]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_COHEBERT + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/"

# 3 spaCy

## 3.1 Download arquivo modelo

https://spacy.io/models/pt

### Função download modelo spaCy

In [ ]:
def downloadSpacy(model_args):
    """
      Realiza o download do arquivo do modelo para o diretório corrente.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
    """
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
        
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Nome arquivo compactado
    NOME_ARQUIVO_MODELO_COMPACTADO = ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    # Url do arquivo
    URL_ARQUIVO_MODELO_COMPACTADO = "https://github.com/explosion/spacy-models/releases/download/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + NOME_ARQUIVO_MODELO_COMPACTADO

    # Realiza o download do arquivo do modelo
    logging.info("Download do arquivo do modelo do spaCy.")
    downloadArquivo(URL_ARQUIVO_MODELO_COMPACTADO, DIRETORIO_COHEBERT + "/" + NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.2 Descompacta o arquivo do modelo

### Função descompacta modelo spaCy

In [ ]:
# Import das bibliotecas.
import tarfile # Biblioteca de descompactação

def descompactaSpacy(model_args):
    """
      Descompacta o arquivo do modelo.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
    """
    
    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    
    # Nome do arquivo a ser descompactado
    NOME_ARQUIVO_MODELO_COMPACTADO = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    logging.info("Descompactando o arquivo do modelo do spaCy.")
    arquivo_tar = tarfile.open(NOME_ARQUIVO_MODELO_COMPACTADO, "r:gz")    
    arquivo_tar.extractall(DIRETORIO_COHEBERT)    
    arquivo_tar.close()
    
    # Apaga o arquivo compactado
    if os.path.isfile(NOME_ARQUIVO_MODELO_COMPACTADO):        
        os.remove(NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.3 Carrega o modelo

### Função carrega modelo spaCy

In [ ]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

def carregaSpacy(model_args):
    """
    Realiza o carregamento do Spacy.
    
    Parâmetros:
      `model_args` - Objeto com os argumentos do modelo.           
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
                  
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Caminho raoz do modelo do spaCy
    DIRETORIO_MODELO_SPACY =  DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY

    # Verifica se o diretório existe
    if os.path.exists(DIRETORIO_MODELO_SPACY) == False:
        # Realiza o download do arquivo modelo do spaCy
        downloadSpacy(model_args)
        # Descompacta o spaCy
        descompactaSpacy(model_args)

    # Diretório completo do spaCy
    DIRETORIO_MODELO_SPACY = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + ARQUIVO_MODELO_SPACY + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/"

    # Carrega o spaCy. Necessário somente "tagger" para encontrar os substantivos
    nlp = spacy.load(DIRETORIO_MODELO_SPACY)
    logging.info("spaCy carregado.")

    # Retorna o spacy carregado
    return nlp 

### Carrega o modelo spaCy


In [ ]:
# Carrega o modelo spaCy
nlp = carregaSpacy(model_args)

INFO:root:Diretório Cohebert criado: COHQUAD_CO_PTBR
INFO:root:Download do arquivo do modelo do spaCy.
INFO:root:Download do arquivo: COHQUAD_CO_PTBR/pt_core_news_lg-3.2.0.tar.gz.


  0%|          | 0.00/577M [00:00<?, ?B/s]

INFO:root:Descompactando o arquivo do modelo do spaCy.
INFO:root:spaCy carregado.


## 3.4 Funções auxiliares spaCy

### getStopwords

Recupera as stopwords do spaCy

In [ ]:
def getStopwords(nlp):
    """
      Recupera as stop words do nlp(Spacy).
    
      Parâmetros:
        `nlp` - Um modelo spaCy carregado.           
    """
    
    spacy_stopwords = nlp.Defaults.stop_words

    return spacy_stopwords 

Lista dos stopwords

In [ ]:
logging.info("Quantidade de stopwords: {}.".format(len(getStopwords(nlp))))

print(getStopwords(nlp))

INFO:root:Quantidade de stopwords: 416.


{'menos', 'próxima', 'pois', 'quinze', 'pelas', 'apoia', 'final', 'nove', 'tua', 'desse', 'deverá', 'pouca', 'este', 'esse', 'tais', 'pegar', 'número', 'era', 'só', 'faço', 'pela', 'custa', 'nesta', 'estiveram', 'você', 'após', 'nesse', 'nos', 'neste', 'sou', 'ali', 'te', 'fez', 'nossos', 'meus', 'no', 'vosso', 'foste', 'pôde', 'porquê', 'aquele', 'à', 'pode', 'minha', 'quê', 'vais', 'último', 'tempo', 'seu', 'também', 'máximo', 'tentaram', 'meu', 'conhecido', 'quieta', 'novo', 'quieto', 'entre', 'algo', 'devem', 'fazemos', 'local', 'cá', 'quais', 'diz', 'estão', 'conselho', 'essa', 'os', 'ora', 'temos', 'fim', 'vinda', 'outros', 'deste', 'estas', 'segundo', 'for', 'são', 'maioria', 'pelo', 'por', 'sistema', 'fomos', 'poderá', 'foram', 'grande', 'pouco', 'teve', 'pelos', 'caminho', 'cinco', 'elas', 'sei', 'tentei', 'estivestes', 'ao', 'ou', 'certamente', 'dezassete', 'valor', 'tanto', 'ambas', 'cada', 'numa', 'nosso', 'quanto', 'dizem', 'dos', 'antes', 'isso', 'além', 'todos', 'sempre'

### getVerbos
Localiza os verbos da sentença

In [ ]:
# Import das bibliotecas.
import spacy   
from spacy.util import filter_spans
from spacy.matcher import Matcher

# (verbo normal como auxilar ou auxilar) + vários verbos auxiliares +verbo principal ou verbo auxiliar
gramaticav1 =  [
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar                                  
                {"POS": "VERB", "OP": "?", "DEP": {"IN": ["ROOT","aux","xcomp","aux:pass"]}},  #verbo normal como auxiliar
                {"POS": "AUX", "OP": "*", "DEP": {"IN": ["aux","xcomp","aux:pass"]}},  #verbo auxiliar   
                {"POS": "VERB", "OP": "+"}, #verbo principal
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop","aux","xcomp","aux:pass"]}},  #verbo auxiliar
               ] 

# verbo auxiliar + verbo normal como auxiliar + conjunção com preposição + verbo
gramaticav2 =  [               
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar                   
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["ROOT"]}},  #verbo principal       
                {"POS": "SCONJ", "OP": "+", "DEP": {"IN": ["mark"]}}, #conjunção com preposição
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["xcomp"]}}, #verbo normal como complementar
               ] 

#Somente verbos auxiliares
gramaticav3 =  [
                {"POS": "AUX", "OP": "?"},  #Verbos auxiliar 
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop"]}},  #Verbos auxiliar de ligação (AUX+(cop))
                {"POS": "ADJ", "OP": "+", "DEP": {"IN": ["ROOT"]}}, 
                {"POS": "AUX", "OP": "?"}  #Verbos auxiliar 
               ] 

matcherv = Matcher(nlp.vocab)
         
matcherv.add("frase verbal", [gramaticav1])
matcherv.add("frase verbal", [gramaticav2])
matcherv.add("frase verbal", [gramaticav3])

#Retorna a Frase Verbal
def getVerbos(periodo):    
  #Processa o período
  doc1 = nlp(periodo.text)
  
  # Chama o mather para encontrar o padrão
  matches = matcherv(doc1)

  padrao = [doc1[start:end] for _, start, end in matches]

  #elimina as repetições e sobreposições
  #return filter_spans(padrao)
  lista1 = filter_spans(padrao)

  # Converte os itens em string
  lista2 = []
  for x in lista1:
      lista2.append(str(x))
  
  return lista2

### getDicPOSQtde

Conta as POS Tagging de uma sentença

In [ ]:
def getDicPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades
  novodic = dict()
  
  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

In [ ]:
def getDicTodasPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades    
  novodic = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ":0, "X": 0}
    
  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

### getDicTodasPOSQtde

Conta as POS Tagging de uma sentença

In [ ]:
def getDicTodasPOSQtde(lista):

  # Dicionário com as tags e quantidades
  conjunto = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ": 0}

  for x in lista:
    valor = conjunto.get(x)
    if valor != None:
      conjunto[x] = valor + 1
    else:
      conjunto[x] = 1

  return conjunto

### getSomaDic

Soma os valores de dicionários com as mesmas chaves.

In [ ]:
from collections import Counter
from functools import reduce

def atualizaValor(a,b):
    a.update(b)
    return a

def getSomaDic(lista):
    
  # Soma os dicionários da lista
  novodic = reduce(atualizaValor, (Counter(dict(x)) for x in lista))
 
  return novodic

### getTokensSentenca

Retorna a lista de tokens da sentenca.

In [ ]:
def getTokensSentenca(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:    
    lista.append(token.text)

  return lista

### getPOSTokensSentenca

Retorna a lista das POS-Tagging dos tokens da sentenca.

In [ ]:
def getPOSTokensSentenca(sentenca):

  # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:    
    lista.append(token.pos_)

  return lista

### getListaTokensPOSSentenca

Retorna duas listas uma com os tokens e a outra com a POS-Tagging dos tokens da sentenca.

In [ ]:
def getListaTokensPOSSentenca(sentenca):
  # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  listatokens = []
  listapos = []

  # Percorre a sentença adicionando os tokens e as POS
  for token in doc:    
    listatokens.append(token.text)
    listapos.append(token.pos_)
    
  return listatokens, listapos

### Tradução das tags

Tags de palavras universal

https://universaldependencies.org/u/pos/

Detalhes das tags em português:
http://www.dbd.puc-rio.br/pergamum/tesesabertas/1412298_2016_completo.pdf

In [ ]:
#dicionário que contêm pos tag universal e suas explicações
palavra_universal_dict = {
  "X"    : "Outro",
  "VERB" : "Verbo ",
  "SYM"  : "Símbolo",
  "CONJ" : "Conjunção",
  "SCONJ": "Conjunção subordinativa",
  "PUNCT": "Pontuação",
  "PROPN": "Nome próprio",
  "PRON" : "Pronome substativo",
  "PART" : "Partícula, morfemas livres",
  "NUM"  : "Numeral",
  "NOUN" : "Substantivo",
  "INTJ" : "Interjeição",
  "DET"  : "Determinante, Artigo e pronomes adjetivos",
  "CCONJ": "Conjunção coordenativa",
  "AUX"  : "Verbo auxiliar",
  "ADV"  : "Advérbio",
  "ADP"  : "Preposição",
  "ADJ"  : "Adjetivo"
}
  
#Explica a POS
def getPOSPalavraUniversalTraduzido(palavra):
  if palavra in palavra_universal_dict.keys():
      traduzido = palavra_universal_dict[palavra]
  else:
      traduzido = "NA" 
  return traduzido

### getSentencaSemStopWord

Retorna uma lista dos tokens sem as stopwords.

In [ ]:
def getSentencaSemStopWord(sentenca, stopwords):

  # Lista dos tokens
  lista = []

  # Percorre os tokens da sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é uma stopword
    if token.lower() not in stopwords:
      lista.append(token)

  # Retorna o documento
  return lista

### getSentencaSalientePOS

Retorna uma lista das palavras do tipo especificado.

In [ ]:
def getSentencaSalientePOS(sentenca, pos, classe_saliente=["NOUN"]):
  
  # Lista dos tokens
  lista = []

  # Percorre a sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é do tipo especificado
    if pos[i] in classe_saliente:
      lista.append(token)

  # Retorna o documento
  return lista

###removeStopWords

Remove as stopwords de um documento ou senteça.

In [ ]:
def removeStopWord(documento, stopwords):
  
  # Remoção das stopwords do documento
  documentoSemStopwords = [palavra for palavra in documento.split() if palavra.lower() not in stopwords]

  # Concatena o documento sem os stopwords
  documento_limpo = " ".join(documentoSemStopwords)

  # Retorna o documento
  return documento_limpo

### getTokensSemStopword

Retira as stopswords de lista de tokens

In [ ]:
def getTokensSemStopword(tokens, spacy_stopwords=getStopwords(nlp)):
    """
      Retira os tokens da lista de tokens tokens que estão na lista de stopword.
      A lista de tokens pode ou não estar dentro de uma outra lista.
    
      Parâmetros:
        `tokens` - Uma lista com os tokens ou uma lista de lista de tokens.
        `spacy_stopwords` - Uma lista com as stopword. 
    """
    
    # Verifica se é uma lista de palavras(str) ou ou uma lista de lista
    if type(tokens[0]) is str:
      lista_tokens = [tokens]
    else:
      lista_tokens = tokens
      
    # Lista de retorno
    lista_tokens_sem_stopwords = []  

    # Percorre a lista de tokens
    for texto in lista_tokens:

      # Lista dos tokens sem as stopwords
      tokens_sem_stopwords = []
      
      # Percorre os tokens    
      for token in texto:
        # Verifica se o toke não está na lista de stopwords para adicionar a nova lista
        if token not in spacy_stopwords:
          tokens_sem_stopwords.append(token)
      
      # Adiciona a lista de tokens sem stopwords na lista de retorno se tiver uma palavra
      if len(tokens_sem_stopwords) != 0:
        lista_tokens_sem_stopwords.append(tokens_sem_stopwords)

    if type(tokens[0]) is str:      
      return lista_tokens_sem_stopwords[0]
    else:
      return lista_tokens_sem_stopwords

### getSentencasTexto

Retorna a lista de tokens de uma lista de textos.

In [ ]:
def getSentencasTexto(textos, nlp = nlp):

  """
     Sentencia um texto ou uma lista de textos.
    
     Parâmetros:
      `textos` - Um texto(str) ou uma lista de textos.
      `nlp` - Modelo spacy carregado.

  """

  # Verifica se é um texto é str ou uma lista de texto
  if type(textos) is str:
    lista_texto = [textos]
  else:
    lista_texto = textos

  # Lista dos tokens
  lista_sentencas = []

  for texto in lista_texto:

    # Sentencia o documento
    doc = nlp(texto)
      
    # Percorre as sentenças do documento
    for sentenca in doc.sents:   

        lista_sentencas.append(str(sentenca))
      
  # Verifica o tipo documento para o tipo de retorno
  if type(textos) is str:
    return lista_sentencas[0]
  else:
    return lista_sentencas

### getSentencasMinusculo

Retorna a lista das sentencas do texto em minúsculo.

In [ ]:
def getSentencasMinusculo(textos):

  """
     Sentencia um texto ou uma lista de textos em minusculo.
    
     Parâmetros:
      `textos` - Um texto(str) ou uma lista de textos.

  """

  # Verifica se é um texto é str ou uma lista de texto
  if type(textos) is str:
    lista_texto = [textos]
  else:
    lista_texto = textos

  # Lista dos tokens
  lista_sentencas = []

  for texto in lista_texto:

    lista_sentencas.append(str(texto).lower())
      
  # Verifica o tipo documento para o tipo de retorno
  if type(textos) is str:
    return lista_sentencas[0]
  else:
    return lista_sentencas

### getTokensTexto

Retorna a lista de tokens do texto.

In [ ]:
def getTokensTexto(textos, nlp = nlp):

  """
     Tokeniza um texto ou uma lista de textos.
    
     Parâmetros:
      `textos` - Um texto(str) ou uma lista de textos.
  """

  # Verifica se é um texto é str ou uma lista de texto
  if type(textos) is str:
    lista_texto = [textos]
  else:
    lista_texto = textos

  # Lista de retorno
  lista_tokens_texto = []

  # Percorre a lista de texto
  for texto in lista_texto:

    # Verifica se o sentenca não foi processado pelo spaCy  
    if type(texto) is not spacy.tokens.doc.Doc:
        # Realiza o parsing no spacy
        doc = nlp(texto)
    else:
        doc = texto

    # Lista dos tokens
    lista_tokens = []

    # Percorre a sentença adicionando os tokens
    for token in doc:    
      lista_tokens.append(token.text)
    
    # Adiciona a lista de tokens na lista de sentenças
    lista_tokens_texto.append(lista_tokens)

  # Verifica o tipo documento para o tipo de retorno
  if type(textos) is str:
    return lista_tokens_texto[0]
  else:
    return lista_tokens_texto

### removerPontuacao

Remove pontuação

In [ ]:
def removerPontuacao(textos):
    
    """https://spacy.io/api/annotation"""

    textos_saida = []

    for texto in textos:
        
        doc = nlp(" ".join(texto)) 

        sentenca = []
        for token in doc:
          if token.pos_ not in ['PUNCT']:
              sentenca.append(token.text)

        if len(sentenca) != 0:
          textos_saida.append(sentenca)

    return textos_saida

### relevantes

Palavras relevantes

In [ ]:
def relevantes(textos, postags_permitidas=['VER', 'AUX', 'NOUN']):
    
    """https://spacy.io/api/annotation"""

    textos_saida = []

    for texto in textos:
        
        doc = nlp(" ".join(texto)) 
      
        sentenca = []
        for token in doc:
          if token.pos_ in postags_permitidas:
              sentenca.append(token.text)

        if len(sentenca) != 0:
          textos_saida.append(sentenca)

    return textos_saida

### lematizacao

Lematização do texto

In [ ]:
def lematizacao(textos, postags_permitidas=['NOUN', 'ADJ', 'VERB', 'ADV']):
    
    """https://spacy.io/api/annotation"""

    textos_saida = []

    for texto in textos:
        doc = nlp(" ".join(texto)) 

        sentenca = []
        for token in doc:
          if token.pos_ in postags_permitidas:
              sentenca.append(token.lemma_)

        if len(sentenca) != 0:
          textos_saida.append(sentenca)

    return textos_saida

### preparaCorpus

In [ ]:
# Import das biblitecas
import pandas as pd
import re
import gensim

def preparaCorpus(textos,                                     
                  sentenciaTexto=False,
                  tornaMinusculo=False,
                  removePontuacao=False, 
                  removeStopwords=False, 
                  bigramas=False, 
                  trigramas=False,
                  somenteRelevante=False,
                  postag_relevante=['VERB', 'AUX', 'NOUN'],
                  lematizar=False,                  
                  postag_lema=['NOUN', 'ADJ', 'VERB', 'ADV']):

    # Verifica se é um textos é str ou uma lista de texto
    if type(textos) is str:
      # Sentencia o texto
      lista_sentencas = [textos]
    else:
      lista_sentencas = textos
    
    # Converte o texto em uma lista de sentencas
    if sentenciaTexto==True:
      lista_sentencas = getSentencasTexto(lista_sentencas)

    # Converte o texto em minúsuclo
    if tornaMinusculo==True:
      lista_sentencas = getSentencasMinusculo(lista_sentencas)
    
    # tokeniza o texto
    lista_sentencas_palavras = getTokensTexto(lista_sentencas)

    # Remove a pontuação 
    if removePontuacao==True:
        lista_sentencas_palavras = removerPontuacao(lista_sentencas_palavras)        

    # Remove as stop words
    if removeStopwords==True:
      lista_sentencas_palavras = getTokensSemStopword(lista_sentencas_palavras)

    # Criar bigramas ou trigramas
    if bigramas==True:
      # Construa os modelos de bigramas
      bigram = gensim.models.Phrases(lista_sentencas_palavras, min_count=5, threshold=100) # max_topicse mais alto menos frases.
      # Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
      bigram_mod = gensim.models.phrases.Phraser(bigram)
      lista_sentencas_palavras = [bigram_mod[doc] for doc in lista_sentencas_palavras]
    
    if trigramas==True:      
      # Construa os modelos de bigramas
      bigram = gensim.models.Phrases(lista_sentencas_palavras, min_count=5, threshold=100) # max_topicse mais alto menos frases.
      # Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
      bigram_mod = gensim.models.phrases.Phraser(bigram)
      # Construa os modelos de trigramas
      trigram = gensim.models.Phrases(bigram[lista_sentencas_palavras], threshold=100)
      # Maneira mais rápida de obter uma frase batida como um trigrama/bigrama    
      trigram_mod = gensim.models.phrases.Phraser(trigram)   
      lista_sentencas_palavras = [trigram_mod[bigram_mod[doc]] for doc in lista_sentencas_palavras]   
    
    # Somente palavras relevantes
    if somenteRelevante==True:      
      lista_sentencas_palavras = relevantes(lista_sentencas_palavras, postags_permitidas=postag_relevante)
    
    # Faça a lematização mantendo apenas para noun, adj, vb, adv
    if lematizar==True:      
      lista_sentencas_palavras = lematizacao(lista_sentencas_palavras, postags_permitidas=postag_lema)

    return lista_sentencas_palavras

# 4 Corpus Específico

## 4.1 Especifica os nomes dos arquivos do corpus

In [ ]:
# Nome do arquivo
NOME_ARQUIVO_CORPUS = "corpus_especifico.csv"
NOME_ARQUIVO_CORPUS_COMPACTADO = "corpus_especifico.zip"

## 4.2 Cria o diretório local para receber os dados

In [ ]:
# Biblioteca para acessar o sistema de arquivos
import os

#Cria o diretório para receber os arquivos Originais e Perturbados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}.".format(dirbase))
else:    
    logging.info("Diretório já existe: {}.".format(dirbase))

INFO:root:Diretório já existe: /content/COHQUAD_CO_PTBR.


## 4.3 Conteúdo do Arquivo do Corpus

In [ ]:
documentos = [
# Pilhas https://pt.wikipedia.org/wiki/Pilha_(inform%C3%A1tica)
['pilha','wikipedia','Em ciência da computação, uma pilha (stack em inglês) é um tipo abstrato de dado e estrutura de dados baseado no princípio de Last In First Out (LIFO), ou seja "o último que entra é o primeiro que sai" caracterizando um empilhamento de dados.'],
['pilha','wikipedia','Pilhas são fundamentalmente compostas por duas operações: push (empilhar) que adiciona um elemento no topo da pilha e pop (desempilhar) que remove o último elemento adicionado.'],
['pilha','wikipedia','Pilhas zamba são usadas extensivamente em cada nível de um sistema de computação moderno.'],
['pilha','wikipedia','Por exemplo, um PC moderno usa pilhas ao nível de arquitetura, as quais são usadas no design básico de um sistema operacional para manipular interrupções e chamadas de função do sistema operacional.'],
['pilha','wikipedia','Entre outros usos, pilhas são usadas para executar uma Máquina virtual java e a própria linguagem Java possui uma classe denominada "Stack", as quais podem ser usadas pelos programadores.'],
['pilha','wikipedia','A pilha é onipresente.'],
['pilha','wikipedia','Um sistema informático baseado em pilha é aquele que armazena a informação temporária basicamente em pilhas, em vez de registradores de hardware da UCP (um sistema baseado em registradores).'],

# Pilha Thomas Cormen
['pilha','Thomas Cormen','As pilhas e filas são conjuntos dinâmicos nos quais o elemento removido do conjunto pela operação DELETE é especificado previamente.'],
['pilha','Thomas Cormen','Em uma pilha, o elemento eliminado do conjunto é o mais recentemente inserido: a pilha implementa uma norma de último a entrar, primeiro a sair, ou LIFO (last-in, first-out).'],
['pilha','Thomas Cormen','De modo semelhante, em uma fila, o elemento eliminado é sempre o que esteve no conjunto pelo tempo mais longo: a fila implementa uma norma de prímeiro a entrar, primeiro a sair, ou FIFO (first-in, first-out).'], 
['pilha','Thomas Cormen','Existem vários modos eficientes de implementar pilhas e filas em um computador.'],
['pilha','Thomas Cormen','Nesta seção, mostraremos como usar um arranjo simples para implementar cada uma dessas estruturas.'],
['pilha','Thomas Cormen','A operação INSERT sobre uma pilha é chamada com frequência PUSH, e a operação DELETE, que não toma um argumento de elemento, é freqientemente chamada POP.'],
['pilha','Thomas Cormen','Esses nomes são alusões a pilhas físicas, como as pilhas de pratos usados em restaurantes.'],
['pilha','Thomas Cormen','A ordem em que os pratos são retirados da pilha é o oposto da ordem em que eles são colocados sobre a pilha e, como conseqúência, apenas o prato do topo está acessível.'],
['pilha','Thomas Cormen','Como mostra a Figura 10.1, podemos implementar uma pilha de no máximo n elementos com um arranjo S[1.. ].'],
['pilha','Thomas Cormen','O arranjo tem um atributo topo[S] que realiza a indexação do elemento inserido mais recentemente.'],
['pilha','Thomas Cormen','A pilha consiste nos elementos S[1..topo[S]], onde s[1] é o elemento na parte inferior da pilha e S[topo[S]] é o elemento na parte superior (ou no topo).'],
['pilha','Thomas Cormen','Quando topo(S] = O, a pilha não contém nenhum elemento e está vazia.'],
['pilha','Thomas Cormen','É possível testar se a pilha está vazia, através da operação de consulta STACK-EMPTY.'],
['pilha','Thomas Cormen','Se uma pilha vazia sofre uma Operação de extração, dizemos que a pilha tem um estouro negativo, que é normalmente um erro.'],
['pilha','Thomas Cormen','Se topo[S] excede n, a pilha tem um estonuro posítivo.'],
['pilha','Thomas Cormen','(Em nossa implementação de pseudocódigo, não nos preocuparemos com o estouro de pilhas.)'], 
['pilha','Thomas Cormen','FIGURA 10.1 Uma implementação de arranjo de uma pilha S.'], 
['pilha','Thomas Cormen','Os elementos da pilha só aparecem nas posições levemente sombreadas.'],
['pilha','Thomas Cormen','(a) A pilha S tem á elementos.'],
['pilha','Thomas Cormen','O elemento do topo é 9.'],
['pilha','Thomas Cormen','(b) A pilha S após as chamadas PUSH(S, 17) e PUSH(S, 3).'],
['pilha','Thomas Cormen','(c) A pilha S após a chamada POP(S) retornou o elemento 3, que é o e lemento mais recentemente inserido na pilha.'],
['pilha','Thomas Cormen','Embora o elemento 3 ainda apareça no arranjo, ele não está mais na pilha; o elemento do topo é o elemento 17.'],
['pilha','Thomas Cormen','Cada uma das operações sobre pilhas pode ser implementada com algumas linhas de código.'],
['pilha','Thomas Cormen','STACK-EMPTY(S)'],
['pilha','Thomas Cormen','1 if topo[s] = O'],
['pilha','Thomas Cormen','2  then return TRUE'],
['pilha','Thomas Cormen','3  else return FALSE'],
['pilha','Thomas Cormen','PUSH(S, x)'],
['pilha','Thomas Cormen','1 topo[S] <- topo[S] + 1'],
['pilha','Thomas Cormen','2 S[topo[S]] <- x'],
['pilha','Thomas Cormen','POP(S)'],
['pilha','Thomas Cormen','1 if STACK-EMPTY(S)'],
['pilha','Thomas Cormen','2  then error \“underflow\"'],
['pilha','Thomas Cormen','3  else topolS] <- topo[S]-1'],
['pilha','Thomas Cormen','4    return S[topo[S] + 1)'],
['pilha','Thomas Cormen','A Figura 10.1 mostra os feitos das operações de modificação PUSH (EMPILHAR) e POP(DESEMPILHAR).'], 
['pilha','Thomas Cormen','Cada uma das três operações sobre pilhas demora o tempo O(1).'],

# Fila https://pt.wikipedia.org/wiki/FIFO
['fila','wikipedia','Em Ciência da Computação, algoritmo de fila simples,FIFO (do inglês: first in, first out, \"primeiro a entrar, primeiro a sair\", "PEPS") ou FCFS (do inglês: first come, first served, "primeiro a chegar, primeiro a ser servido") é um algoritmo de escalonamento para estruturas de dados do tipo fila.'],
['fila','wikipedia','Apresenta o seguinte critério: o primeiro elemento a ser retirado é o primeiro que tiver sido inserido, é um algoritmo de escalonamento não preemptivo que entrega a CPU os processos pela ordem de chegada.'],
['fila','wikipedia','Ele executa o processo como um todo do inicio ao fim não interrompendo o processo executado até ser finalizado, então quando um novo processo chega e existe um ainda em execução ele vai para uma fila de espera.'],
['fila','wikipedia','Esta fila de espera nada mais é do que uma fila que organiza os processos que chegam até eles serem atendidos pela CPU.'],
['fila','wikipedia','Neste escalonamento todos os processos tendem a serem atendidos (por isso evita o fenômeno do starvation) ao menos que um processo possua um erro ou loop infinito.'],
['fila','wikipedia','O loop infinito irá parar a máquina, pois com o FIFO não terá como dar continuidade a execução dos processos que estão aguardando na fila de espera.'],
['fila','wikipedia','O algoritmo FIFO não garante um tempo de resposta rápido pois é extremamente sensível a ordem de chegada de cada processo e dos antecessores (se existirem) e se processos que tendem a demorar mais tempo chegarem primeiro o tempo médio de espera e o turnaround acabam sendo aumentados.'],
['fila','wikipedia','Pelo critério do primeiro a entrar é o primeiro a ser servido, faz o agendamento de tarefas do sistema operacional dando a cada processo tempo de CPU na ordem em que as demandas são feitas.'],
['fila','wikipedia','O oposto de FIFO é LIFO (Last-In, First-Out), que significa "o último a entrar é o primeiro a sair", aonde a entrada mais recente, ou o topo da pilha de processos, é processado primeiro.[4].'],
['fila','wikipedia','Já uma fila prioritária não é nem FIFO, nem LIFO, mas pode adotar comportamento similar temporariamente, ou mesmo por padrão.'],
['fila','wikipedia','As listas são amplamente utilizadas em programação para implementar filas de espera.'],
['fila','wikipedia','Em uma fila de tipo FIFO os elementos vão sendo colocados na fila e retirados (ou processados) por ordem de chegada.'],
['fila','wikipedia','A ideia fundamental da fila é que só podemos inserir um novo elemento no final da fila e só podemos retirar o elemento do início.'],
['fila','wikipedia','É vantajoso por ser o mais simples entre os processos de escalonamento; e todos os processos tendem a serem atendidos.'],
['fila','wikipedia','Dentre as desvantagens estão: muito sensível a ordem de chegada; se processos maiores chegarem primeiro aumentarão o tempo médio de espera; nãoo garante um tempo de resposta rápido; não é eficiente em sistemas de tempo compartilhado; e não é eficiente em sistemas em tempo real.'],
['fila','wikipedia','FIFO são comumente usados em circuitos eletrônicos de buffer e controle de fluxo, que vai desde o hardware até o software.'],
['fila','wikipedia','Na forma de um hardware o FIFO consiste basicamente de um conjunto de ler e escrever ponteiros, armazenamento e lógica de controle.'],
['fila','wikipedia','Armazenamento pode ser SRAM, flip-flops, fechos ou qualquer outra forma adequada de armazenamento.'],
['fila','wikipedia','Para o FIFO, de tamanho não trivial, uma SRAM de porta dupla geralmente é utilizada quando uma porta é usada para a escrita e a outra para leitura.'],
['fila','wikipedia','O FIFO síncrono aonde o mesmo clock é usado para leitura e escrita.'],
['fila','wikipedia','Um FIFO assíncrono utiliza diferentes relógios para leitura e escrita.'],
['fila','wikipedia','Uma aplicação comum de um FIFO assíncrono utiliza um código de Gray (código binário refletido), ou qualquer unidade de código a distância, para a ler e escrever os ponteiros para garantir a geração de bandeira confiável.'],
['fila','wikipedia','Uma nota mais preocupante é que se deve necessariamente usar a aritmética de ponteiro para gerar bandeiras para implementações assíncronas FIFO.'],
['fila','wikipedia','Por outro lado, pode-se usar a abordagem de um balde "de fuga" ou a aritmética de ponteiro para gerar bandeiras nas implementações síncronas FIFO.'],

# Fila Thomas Cormen
['fila','Thomas Cormen','Chamamos a operação INSERT sobre uma fila de ENQUEUE (ENFILEIRAR), e também a operação DELETE de DEQUEUE (DESINFILEIRAR); como a operação sobre pilhas POP, DEQUEUE não tem nenhum argumento de elemento.'],
['fila','Thomas Cormen','A propriedade FIFO de uma fila faz com que ela opere como uma fileira de pessoas no posto de atendimento da previdência social.'],
['fila','Thomas Cormen','A fila tem um início (ou cabeça) e um fim (ou cauda).'],
['fila','Thomas Cormen','Quando um elemento é colocado na fila, ele ocupa seu lugar no fim da fila, como um aluno recém-chegado que ocupa um lugar no final da fileira.'],
['fila','Thomas Cormen','O elemento retirado da fila é sempre aquele que está no início da fila, como o aluno que se encontra no começo da fileira e que esperou por mais tempo.'],
['fila','Thomas Cormen','(Felizmente, não temos de nos preocupar com a possibilidade de elementos computacionais \“furarem\” a fila.)'],
['fila','Thomas Cormen','A Figura 10.2 mostra um modo de implementar uma fila de no máximo n - 1 elementos usando um arranjo O[1..n].'],
['fila','Thomas Cormen','A fila tem um atributo ínício[Q] que indexa ou aponta para seu início.'],
['fila','Thomas Cormen','O atributo fím[Q] realiza a indexação da próxima posição na qual um elemento recém chegado será inserido na fila.'],
['fila','Thomas Cormen','Os elementos na fila estão nas posições início[Q), início[Q]+1, ... fim[Q]-1, onde “retornamos”, no sentido de que a posição 1 segue imediatamente a posição n em uma ordem circular.'],
['fila','Thomas Cormen','Quando início[Q] = fim[Q]), a fila está vazia.'],
['fila','Thomas Cormen','Inicialmente, temos início[Q] = fim[Q] = 1.'],
['fila','Thomas Cormen','Quando a fila está vazia, uma tentativa de retirar um elemento da fila provoca o estouro negativo da fila.'],
['fila','Thomas Cormen','Quando início[Q] = fim[Q] + 1, a fila está cheia, e uma tentativa de colocar um elemento na fila provoca o estouro positivo da fila.'],
['fila','Thomas Cormen','FIGURA 10.2 Uma fila implementada com a utilização de um arranjo Q[1..12).'],
['fila','Thomas Cormen','Os elementos da fila aparecem apenas nas posições levemente sombreadas.'],
['fila','Thomas Cormen','(a) A filatem 5 elementos, nas localizações O[7..11].'], 
['fila','Thomas Cormen','(b) A configuração da fila depois das chamadas ENQUEUE(Q, 17), ENQUEUE(Q, 3) e ENQUEUE(O, 5).'],
['fila','Thomas Cormen','A configuração da fila depois da chamada DEQUEUE(Q) retorna o valor de chave 15 que se encontrava anteriormente no início da fila.'],
['fila','Thomas Cormen','O novo início tem a chave 6'],
['fila','Thomas Cormen','Em nossos procedimentos ENQUEUE e DEQUEUE, a verificação de erros de estouro negativo (underfiow) € estouro positivo (overflow) foi omitida.'],
['fila','Thomas Cormen','O Exercício 10.1-4 lhe pede para fornecer o código que efetua a verificação dessas duas condições de erro.'],
['fila','Thomas Cormen','ENQUEUE(Q, x)'],
['fila','Thomas Cormen','1 Q[fim[Q] = x'],
['fila','Thomas Cormen','2 iffim[Q] = comprimento[Q]'],
['fila','Thomas Cormen','3 then fim[Q] = 1'],
['fila','Thomas Cormen','4 else fim[Q] = fim[Q]+1'],
['fila','Thomas Cormen','DEQUEUE(Q)'],
['fila','Thomas Cormen','1 x = OQlinício[Q]]'],
['fila','Thomas Cormen','2 if início[Q] == comprimento[Q]'],
['fila','Thomas Cormen','3 then início[Q] = 1'],
['fila','Thomas Cormen','4  else início[Q] & início[Q] + 1'],
['fila','Thomas Cormen','5 return x'],
['fila','Thomas Cormen','A Figura 10.2 mostra os efeitos das operações ENQUEUE e DEQUEUE.'],
['fila','Thomas Cormen','Cada operação demora o tempo O(1).']
]


print("Quantidade de documentos:", len(documentos))

Quantidade de documentos: 104


## 4.5 Cria o arquivo do corpus

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Cria o dataframe da lista
df_lista_sentencas = pd.DataFrame(documentos, columns = ["topico","fonte","sentenca"])
 
df_lista_sentencas.to_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_CORPUS, sep=";", index=False)

## 4.6 Compacta e copia o arquivo perturbado para uma pasta do GoogleDrive

Compacta o arquivo gerado da comparação para facilitar o envio para o GoogleDrive

In [ ]:
!zip -o -q -j "$DIRETORIO_LOCAL$NOME_ARQUIVO_CORPUS_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_CORPUS"

logging.info("Terminei compactação.")

INFO:root:Terminei compactação.


Copia o arquivo compactado e os arquivos individuais para o GoogleDrive

In [ ]:
# Import das bibliotecas.
import os
import datetime

# Se estiver executando no Google Colaboratory
if IN_COLAB:
     
    # Copia o arquivo perturbado
    !cp "$DIRETORIO_LOCAL$NOME_ARQUIVO_CORPUS_COMPACTADO" "$DIRETORIO_DRIVE"
    
    logging.info("Terminei a cópia.")

INFO:root:Terminei a cópia.


## 4.7 Carrega os dados

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
df_corpus = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_CORPUS, sep=";", encoding="UTF-8")

print(len(df_corpus))

104


In [ ]:
df_corpus.sample(5)

,topico,fonte,sentenca
74,fila,Thomas Cormen,"(Felizmente, não temos de nos preocupar com a ..."
36,pilha,Thomas Cormen,1 topo[S] <- topo[S] + 1
30,pilha,Thomas Cormen,Cada uma das operações sobre pilhas pode ser i...
24,pilha,Thomas Cormen,Os elementos da pilha só aparecem nas posições...
17,pilha,Thomas Cormen,"A pilha consiste nos elementos S[1..topo[S]], ..."


# 5 Finalização

## 5.1 Tempo final de processamento



In [ ]:
# Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempo_total_processamento))


  Tempo processamento:  0:02:33 (h:mm:ss)
